## RANDOM FOREST


In [28]:
import numpy as np
import pandas as pd
df_voting=pd.read_csv("/Users/noshitha/Downloads/hw3/datasets/hw3_house_votes_84.csv")

In [29]:
df_voting.head()

,#handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-adi,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,1,2,1,2,2,2,1,1,1,2,0,2,2,2,1,2,1
1,1,2,1,2,2,2,1,1,1,1,1,2,2,2,1,0,1
2,0,2,2,0,2,2,1,1,1,1,2,1,2,2,1,1,0
3,1,2,2,1,0,2,1,1,1,1,2,1,2,1,1,2,0
4,2,2,2,1,2,2,1,1,1,1,2,0,2,2,2,2,0


In [30]:
from sklearn.utils import shuffle
df_voting_shuffle = shuffle(df_voting)

In [31]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_voting_shuffle, test_size=0.2, random_state=34)

# Shapes of the training and testing sets
print("\nTraining Set Shape:", train_df.shape)
print("Testing Set Shape:", test_df.shape)


Training Set Shape: (348, 17)
Testing Set Shape: (87, 17)


## Algorithm

In [45]:
import numpy as np

def decision_tree(X_train, y_train, max_depth, current_depth=0):
    # If all instances belong to the same class or max depth is reached
    if len(set(y_train)) == 1 or current_depth == max_depth:
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        return {"class_label": majority_class}
    
    # Calculate information gain for all attributes
    info_gain_values = [information_gain(y_train, X_train[attr]) for attr in X_train.columns]
    best_attribute_index = np.argmax(info_gain_values) # Index of attribute with highest information gain
    best_attribute = X_train.columns[best_attribute_index]
    
    # Create a dictionary for the current decision node
    node = {"attribute": best_attribute, "leaf": {}}
    
    print("Depth: ",current_depth)
    
    # Find the average of the attribute's values
    threshold = np.mean(X_train[best_attribute])

    # Partition the data based on the threshold
    left_partition_indices = X_train[X_train[best_attribute] <= threshold].index
    right_partition_indices = X_train[X_train[best_attribute] > threshold].index

    # Recursively create left and right subtrees
    left_subtree = decision_tree(X_train.loc[left_partition_indices], y_train.loc[left_partition_indices], max_depth, current_depth + 1)
    right_subtree = decision_tree(X_train.loc[right_partition_indices], y_train.loc[right_partition_indices], max_depth, current_depth + 1)

    node["leaf"]["<= " + str(threshold)] = left_subtree
    node["leaf"]["> " + str(threshold)] = right_subtree

    
    return node

def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def information_gain(y, x):
    parent_entropy = entropy(y)
    info_a = 0
    for value in set(x):
        partition_indices = x[x == value].index
        partition_entropy = entropy(y[partition_indices])
        info_a += len(partition_indices) / len(x) * partition_entropy
    gain_a = parent_entropy - info_a
    return gain_a



In [46]:
def predict_instance(tree, instance):
    attribute = next(iter(tree.keys()))  # Get the attribute of the current node
    if attribute not in instance or instance[attribute] not in tree[attribute]:
        # Handle unknown attribute values or missing branches
        if 'default' in tree["leaf"]:
            return tree["leaf"]["default"]
        else:
            # Handle this case according to your application requirements
            # For example, you can return None or raise an exception
            return None
    
    if isinstance(instance[attribute], (int, float)):
        # If the attribute is numeric, check for a threshold
        if instance[attribute] <= list(tree[attribute].keys())[0]:
            # Recur for the left subtree
            return predict_instance(tree[attribute][list(tree[attribute].keys())[0]], instance)
        else:
            # Recur for the right subtree
            return predict_instance(tree[attribute][list(tree[attribute].keys())[1]], instance)
    else:
        # Recur for the subtree corresponding to the attribute value
        return predict_instance(tree[attribute][instance[attribute]], instance)

        
def predict(tree, X):
    predictions = []
    for i in range(len(X)):
        predictions.append(predict_instance(tree, X.iloc[i]))
    return predictions

def accuracy(y_true, y_pred):
    correct = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    return correct / len(y_true) 

In [47]:
tree = decision_tree(X_train, y_train, 5,0)

Depth:  0
Depth:  1
Depth:  2
Depth:  3
Depth:  4
Depth:  4
Depth:  1
Depth:  2
Depth:  3
Depth:  4
Depth:  3
Depth:  4
Depth:  2
Depth:  3
Depth:  4
Depth:  4
Depth:  3
Depth:  4


In [48]:

# Predict Labels
y_pred = predict(tree, X_test)

# Calculate Accuracy
acc = accuracy(y_test, y_pred)
print("Decision Tree Accuracy:", acc)

Decision Tree Accuracy: 0.0


In [26]:
# Bootstrap Dataset Generation
def generate_bootstrap_dataset(X_train, y_train):
    n_samples = len(X_train)
    indices = np.random.choice(n_samples, n_samples, replace=True)
    X_bootstrap = X_train.iloc[indices]
    y_bootstrap = y_train.iloc[indices]
    return X_bootstrap, y_bootstrap

# Random Attribute Selection
def select_random_attributes(X_train, m):
    n_attributes = X_train.shape[1]
    selected_indices = np.random.choice(n_attributes, m, replace=False)
    return selected_indices.tolist()

# Majority Voting Mechanism
def majority_voting(predictions):
    vote_counts = Counter(predictions)
    predicted_class = vote_counts.most_common(1)[0][0]
    return predicted_class

# Stratified Cross-Validation
def stratified_cross_validation(X, y, n_splits, n_trees, max_depth):
    n_samples = X.shape[0]
    fold_size = n_samples // n_splits
    
    accuracies = []
    for i in range(n_splits):
        start_idx = i * fold_size
        end_idx = (i + 1) * fold_size if i < n_splits - 1 else n_samples
        
        X_test_fold = X.iloc[start_idx:end_idx]
        y_test_fold = y.iloc[start_idx:end_idx]
        
        X_train_fold = pd.concat([X.iloc[:start_idx], X.iloc[end_idx:]])
        y_train_fold = pd.concat([y.iloc[:start_idx], y.iloc[end_idx:]])
        
        # Construct Random Forest
        forest = []
        for i in range(n_trees):
            print("Tree:",i)
            X_bootstrap, y_bootstrap = generate_bootstrap_dataset(X_train_fold, y_train_fold)
            tree = decision_tree(X_bootstrap, y_bootstrap, max_depth)
            forest.append(tree)
        
        # Predict Labels
        y_pred = [predict(tree, X_test_fold) for tree in forest]
        y_pred_combined = [majority_voting(predictions) for predictions in zip(*y_pred)]
        
        # Calculate Accuracy
        acc = accuracy(y_test_fold, y_pred_combined)
        accuracies.append(acc)
    
    avg_accuracy = np.mean(accuracies)
    return avg_accuracy

In [27]:
from sklearn.model_selection import train_test_split

# Assuming df_voting contains your dataset with features and target variable
# Features are all columns except the last one, target variable is the last column
X = df_voting.iloc[:, :-1]  # Features
y = df_voting.iloc[:, -1]   # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

# Shapes of the training and testing sets
print("\nTraining Set Shape:", X_train.shape)
print("Testing Set Shape:", X_test.shape)

tree = decision_tree(X_train, y_train, max_depth=5)
y_pred= predict(tree, X_test)
# Calculate Accuracy
acc = accuracy(y_test, y_pred)
print("Decision Tree Accuracy:", acc)



Training Set Shape: (348, 16)
Testing Set Shape: (87, 16)
Depth:  0
Depth:  1
Depth:  2
Depth:  3
Depth:  4
Depth:  4
Depth:  1
Depth:  2
Depth:  3
Depth:  4
Depth:  3
Depth:  4
Depth:  2
Depth:  3
Depth:  4
Depth:  4
Depth:  3
Depth:  4


KeyError: 'default'

In [ ]:


# Evaluate Impact of Number of Trees
n_trees_values = [5, 10, 15, 20]  # Example values for number of trees
avg_accuracies = []

for n_trees in n_trees_values:
    avg_accuracy = stratified_cross_validation(X_train, y_train, n_splits=10, n_trees=n_trees, max_depth=5)
    avg_accuracies.append(avg_accuracy)

# Print Average Accuracies
for n_trees, avg_accuracy in zip(n_trees_values, avg_accuracies):
    print("n_tree_value:",n_trees)
    print(f"Number of Trees: {n_trees}, Average Accuracy: {avg_accuracy}")

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df_voting contains your dataset with features and target variable
# Features are all columns except the last one, target variable is the last column
X = df_voting.iloc[:, :-1]  # Features
y = df_voting.iloc[:, -1]   # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

# Shapes of the training and testing sets
print("\nTraining Set Shape:", X_train.shape)
print("Testing Set Shape:", X_test.shape)
# Decision Tree Construction
tree = decision_tree(X_train, y_train, max_depth=5)

# Predict Labels
y_pred = predict(tree, X_test)

# Calculate Accuracy
acc = accuracy(y_test, y_pred)
print("Decision Tree Accuracy:", acc)


In [ ]:
import numpy as np

def decision_tree(X_train, y_train, max_depth=None):
    if max_depth is not None and max_depth <= 0:
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        print("Reached maximum depth or no more depth left. Returning majority class:", majority_class)
        return {"class_label": majority_class}

    # Randomly select attributes for splitting
    selected_attributes = np.random.choice(X_train.columns, len(X_train.columns), replace=False)
    #print("Selected attributes:", selected_attributes)
    info_gain_values = [information_gain(y_train, X_train[attr]) for attr in selected_attributes]
    best_attribute = selected_attributes[np.argmax(info_gain_values)]
    print("Best attribute:", best_attribute)
    node = {"attribute": best_attribute, "leaf": {}}
    attribute_values = sorted(set(X_train[best_attribute]))

    thresholds = [(attribute_values[i] + attribute_values[i+1]) / 2 for i in range(len(attribute_values) - 1)]

    for threshold in thresholds:
        partition_left = X_train[X_train[best_attribute] <= threshold]
        partition_right = X_train[X_train[best_attribute] > threshold]

        if partition_left.empty or partition_right.empty:
            majority_class = y_train.value_counts().idxmax()
            print("Partitioned data has empty partitions. Returning majority class:", majority_class)
            node["leaf"][threshold] = {"class_label": majority_class}
        else:
            print("Partitioning data with threshold:", threshold)
            subtree_left = decision_tree(partition_left, y_train[partition_left.index], max_depth - 1)
            subtree_right = decision_tree(partition_right, y_train[partition_right.index], max_depth - 1)
            node["leaf"][threshold] = {"<= threshold": subtree_left, "> threshold": subtree_right}
    return node

def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def information_gain(y, x):
    parent_entropy = entropy(y)
    info_a = 0
    for value in set(x):
        partition_indices = x[x == value].index
        partition_entropy = entropy(y[partition_indices])
        info_a += len(partition_indices) / len(x) * partition_entropy
    gain_a = parent_entropy - info_a
    return gain_a


In [ ]:
def create_bootstrap_datasets(train_df, num_datasets):
    bootstrap_datasets = []
    num_instances = len(train_df)

    for _ in range(num_datasets):
        # Sampling with replacement
        bootstrap_sample_indices = np.random.choice(num_instances, num_instances, replace=True)
        bootstrap_sample = train_df.iloc[bootstrap_sample_indices]
        bootstrap_datasets.append(bootstrap_sample)

    return bootstrap_datasets

def fit_random_forest(bootstrap_datasets, num_trees=10, max_features=None):
    random_forest = []
    num_instances = len(bootstrap_datasets[0])  # Assuming all datasets have the same length
    num_features = len(bootstrap_datasets[0].columns) - 1  # Excluding the class column

    for tree_index in range(num_trees):
        print("Training tree:", tree_index + 1)
        # Randomly select features if max_features is specified
        selected_features = list(np.random.choice(num_features, max_features, replace=False)) if max_features else list(range(num_features))
        print("Selected features for tree:", selected_features)
        
        # Train a decision tree on each bootstrap dataset
        trees = []
        for dataset_index, dataset in enumerate(bootstrap_datasets):
            X_train = dataset.drop(columns=["class"])
            y_train = dataset["class"]
            print("Training tree", tree_index + 1, "on dataset", dataset_index + 1)
            tree = decision_tree(X_train, y_train, max_depth=2)
            trees.append(tree)
        
        random_forest.append(trees)

    return random_forest

def predict_random_forest(X_test, random_forest):
    predictions = []
    for tree in random_forest:
        predictions.append(predict(X_test, tree))
    
    # Perform majority voting
    majority_votes = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=np.array(predictions))
    return majority_votes


In [ ]:

# Example usage:
num_datasets = 10  # Number of bootstrap datasets to create
bootstrap_datasets = create_bootstrap_datasets(train_df, num_datasets)
print(train_df.columns)
random_forest = fit_random_forest(bootstrap_datasets)

# Predictions
test_X = test_df.drop(columns=["class"])
predictions = predict_random_forest(test_X, random_forest)


In [ ]:
import pandas as pd

def check_same_class(y_train):
    # If all instances belong to the same class
    if len(set(y_train['class'])) == 1:
        return {"class_label": y_train['class'].iloc[0]}
    else:
        return {"not_same_class": True}

# Example usage
y_train = pd.DataFrame({'class': [0, 0, 0, 0, 1, 1, 1, 1]})
check=len(set(y_train['class']))
print(check)
print(set(y_train['class']))
result = check_same_class(y_train)
print(result)


In [ ]:
import numpy as np

def decision_tree(X_train, y_train, remaining_attributes, m=None):
    if len(set(y_train)) == 1:
        return {"class_label": y_train.iloc[0]}
    
    if not remaining_attributes:
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        return {"class_label": majority_class}
    
    if m is None:
        m = int(np.sqrt(len(remaining_attributes)))
    
    # Randomly select 'm' attributes from the remaining attributes
    selected_attributes = np.random.choice(remaining_attributes, m, replace=False)
    
    info_gain_values = [information_gain(y_train, X_train[attr]) for attr in selected_attributes]
    best_attribute = selected_attributes[np.argmax(info_gain_values)]
    remaining_attributes_copy = remaining_attributes.copy()
    remaining_attributes_copy.remove(best_attribute)
    remaining_attributes = remaining_attributes_copy.copy()
    
    node = {"attribute": best_attribute, "leaf": {}}
    attribute_values = sorted(set(X_train[best_attribute]))
    
    thresholds = [(attribute_values[i] + attribute_values[i+1]) / 2 for i in range(len(attribute_values) - 1)]
    
    for threshold in thresholds:
        partition_left = X_train[X_train[best_attribute] <= threshold]
        partition_right = X_train[X_train[best_attribute] > threshold]

        if partition_left.empty or partition_right.empty:
            majority_class = y_train.value_counts().idxmax()
            node["leaf"][threshold] = {"class_label": majority_class}
        else:
            subtree_left = decision_tree(partition_left, y_train[partition_left.index], remaining_attributes, m)
            subtree_right = decision_tree(partition_right, y_train[partition_right.index], remaining_attributes, m)
            node["leaf"][threshold] = {"<= threshold": subtree_left, "> threshold": subtree_right}
            
    return node

def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def information_gain(y, x):
    parent_entropy = entropy(y)
    info_a = 0
    for value in set(x):
        partition_indices = x[x == value].index
        partition_entropy = entropy(y[partition_indices])
        info_a += len(partition_indices) / len(x) * partition_entropy
    gain_a = parent_entropy - info_a
    return gain_a


In [ ]:
def predict_single(x, tree):
    if 'class_label' in tree:
        # Single data point
        return tree['class_label']
    else:
        value = x[tree['attribute']]
        if value in tree['leaf']:
            # leaf has only class label or a nested dictionary
            next_node = tree['leaf'][value]
            if 'class_label' in next_node:
                return next_node['class_label']
            else:
                return predict_single(x, next_node)
        else:
            return list(tree['leaf'].values())[0]


def predict(X, tree):
    return [predict_single(x, tree) for i, x in X.iterrows()]


In [ ]:
import numpy as np

def create_bootstrap_datasets(train_df, num_datasets):
    bootstrap_datasets = []
    num_instances = len(train_df)

    for _ in range(num_datasets):
        # Sampling with replacement
        bootstrap_sample_indices = np.random.choice(num_instances, num_instances, replace=True)
        bootstrap_sample = train_df.iloc[bootstrap_sample_indices]
        bootstrap_datasets.append(bootstrap_sample)

    return bootstrap_datasets

# Example usage:
num_datasets = 10  # Number of bootstrap datasets to create
bootstrap_datasets = create_bootstrap_datasets(train_df, num_datasets)

# Check the shapes of the bootstrap datasets
for i, dataset in enumerate(bootstrap_datasets):
    print(f"Bootstrap Dataset {i+1} Shape:", dataset.shape)


## Decision Tree using info-gain

In [ ]:
import numpy as np

def decision_tree(X_train, y_train, remaining_attributes, m):
    # If all instances belong to the same class
    if len(set(y_train)) == 1:
        return {"class_label": y_train.iloc[0]}
    if not remaining_attributes:
        # If no more attributes left
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        return {"class_label": majority_class}
    
    # Randomly select m attributes
    selected_attributes = np.random.choice(remaining_attributes, size=min(m, len(remaining_attributes)), replace=False)
    
    info_gain_values = [information_gain(y_train, X_train[attr]) for attr in selected_attributes]
    best_attribute = selected_attributes[np.argmax(info_gain_values)] # Highest info gain attribute
    remaining_attributes_copy = remaining_attributes.copy()  # Make a copy
    remaining_attributes_copy.remove(best_attribute) # L(attr)-Selected(attr)
    remaining_attributes = remaining_attributes_copy.copy()  # Update remaining_attributes
    
    # Create a dictionary for the current decision node
    node = {"attribute": best_attribute, "leaf": {}}
    # Create a list of values for the selected attribute
    attribute_values = set(X_train[best_attribute])

    # For each attribute value, create sub-trees and edges
    for value in attribute_values:
        # Partition - based on the attribute value
        partition = X_train[X_train[best_attribute] == value]

        if partition.empty:
            # If partition is empty, leaf node = node with majority class
            majority_class = y_train.value_counts().idxmax()
            node["leaf"][value] = {"class_label": majority_class}
        else:
            # Recursively create a sub-tree
            subtree = decision_tree(partition, y_train[partition.index], remaining_attributes, m)
            node["leaf"][value] = subtree
    return node


In [ ]:
def decision_tree(X_train, y_train, attributes, m):
    if len(set(y_train)) == 1:
        return {"class_label": y_train.iloc[0]}
    if not attributes:
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        return {"class_label": majority_class}

        # Ensure that 'attributes' is a list of attribute names
    attributes = list(train_df.columns[:-1])

    # Call the function with the corrected 'attributes'
    tree = decision_tree(bootstrap_df, attributes, target, m)

    
    selected_attributes = np.random.choice(attributes, size=min(m, len(attributes)), replace=False)
    info_gain_values = [information_gain(y_train, X_train[attr]) for attr in selected_attributes]
    best_attribute = selected_attributes[np.argmax(info_gain_values)]
    
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]
    
    node = {"attribute": best_attribute, "leaf": {}}
    attribute_values = set(X_train[best_attribute])
    for value in attribute_values:
        partition = X_train[X_train[best_attribute] == value]
        if partition.empty:
            majority_class = y_train.value_counts().idxmax()
            node["leaf"][value] = {"class_label": majority_class}
        else:
            subtree = decision_tree(partition, y_train[partition.index], remaining_attributes, m)
            node["leaf"][value] = subtree
    return node

def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def information_gain(y, x):
    parent_entropy = entropy(y)
    # InfoA(D)
    info_a = 0
    for value in set(x):
        partition_indices = x[x == value].index
        partition_entropy = entropy(y[partition_indices])
        info_a += len(partition_indices) / len(x) * partition_entropy
    # GainA(D)
    gain_a = parent_entropy - info_a
    return gain_a

In [ ]:
def predict_single(x, tree):
    if 'class_label' in tree:
        # Single data point
        return tree['class_label']
    else:
        value = x[tree['attribute']]
        if value in tree['leaf']:
            # leaf has only class label or a nested dictionary
            next_node = tree['leaf'][value]
            if 'class_label' in next_node:
                return next_node['class_label']
            else:
                return predict_single(x, next_node)
        else:
            return list(tree['leaf'].values())[0]

def predict(X, tree):
    return [predict_single(x, tree) for i, x in X.iterrows()]

In [ ]:
def random_forest_predict(X, forest):
    predictions = []
    for tree in forest:
        predictions.append(predict(X, tree))
    # Transpose predictions to get each tree's prediction for each instance
    predictions = np.array(predictions).T
    # Use majority voting to get final prediction for each instance
    final_predictions = [np.bincount(row).argmax() for row in predictions]
    return final_predictions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# Function to calculate accuracy
def calculate_accuracy(predictions, actual):
    return np.sum(predictions == actual) / len(actual)

def create_bootstrap_dataset(df):
    return df.sample(n=len(df), replace=True)

def run_random_forest(df, attributes, target, num_trees=100, num_iterations=100):
    forest = []
    for i in range(num_trees):
        bootstrap_df = create_bootstrap_dataset(df)
        tree = decision_tree(bootstrap_df, attributes, target,m)
        forest.append(tree)
    return forest

def stratified_cross_validation(df, attributes, target, num_trees=100, num_folds=10):
    fold_size = len(df) // num_folds
    accuracies = []
    for i in range(num_folds):
        start_index = i * fold_size
        end_index = start_index + fold_size if i < num_folds - 1 else len(df)
        test_df = df.iloc[start_index:end_index]
        train_df = pd.concat([df.iloc[:start_index], df.iloc[end_index:]])
        
        # Train random forest on training set
        forest = run_random_forest(train_df, attributes, target, num_trees)
        
        # Make predictions on test set
        X_test, y_test = test_df[attributes], test_df[target]
        predictions = random_forest_predict(X_test, forest)
        
        # Calculate accuracy
        accuracy = calculate_accuracy(predictions, y_test)
        accuracies.append(accuracy)
    return np.mean(accuracies)

In [ ]:
df_voting_shuffle.head()

In [ ]:
attributes = df_voting_shuffle.columns[:-1].tolist()
target = df_voting_shuffle['class']
train_df, test_df = train_test_split(df_voting_shuffle, test_size=0.2, random_state=34)

In [ ]:
print(attributes)

In [ ]:
 # Split the DataFrame into features and target for training and testing sets
X_train, y_train = train_df[attributes], train_df['class']
X_test, y_test = test_df[attributes], test_df['class']

In [ ]:
# Build the decision tree using the training set

m = int(np.sqrt(len(attributes)))

root_node = decision_tree(X_train, y_train, attributes,m)

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(predictions, actual):
    return np.sum(predictions == actual) / len(actual)

# Make predictions on the training set
predictions_train = predict(X_train, root_node)

# Make predictions on the test set
predictions_test = predict(X_test, root_node)

# Evaluate the accuracy on the training set
accuracy_train = calculate_accuracy(predictions_train, y_train)
print("Accuracy on Training Set:", accuracy_train)

# Evaluate the accuracy on the test set
accuracy_test = calculate_accuracy(predictions_test, y_test)
print("Accuracy on Test Set:", accuracy_test)


In [ ]:

# Evaluate the impact of the number of trees
num_trees_list = [10, 50, 100, 200]
for num_trees in num_trees_list:
    mean_accuracy = stratified_cross_validation(df_voting_shuffle, attributes, 'class', num_trees)
    print("Number of Trees:", num_trees)
    print("Mean Accuracy:", mean_accuracy)


In [ ]:
# Run the experiment
attributes = df_voting_shuffle.columns[:-1].tolist()
target = 'class'
df_voting_shuffle = shuffle(df_voting_shuffle, random_state=np.random.randint(1, 1000))  # Shuffle once initially

accuracy_train, accuracy_test = run_decision_tree(df_voting_shuffle, attributes, target)


In [ ]:
mean_accuracy_train = np.mean(accuracy_train)
mean_accuracy_test = np.mean(accuracy_test)

# Print or use the values as needed
print("Mean Training Accuracy:", mean_accuracy_train)
print("Mean Testing Accuracy:", mean_accuracy_test)

In [ ]:

std_dev_accuracy_train = np.std(accuracy_train)
std_dev_accuracy_test = np.std(accuracy_test)

print("Standard Deviation Training Accuracy:", std_dev_accuracy_train)
print("Standard Deviation Testing Accuracy:", std_dev_accuracy_test)

In [ ]:

# Plot histograms
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(accuracy_train, bins=20, edgecolor='black')
plt.title('Accuracy Distribution on Training Set on info gain')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')


plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 2)
plt.hist(accuracy_test, bins=20, edgecolor='black')
plt.title('Accuracy Distribution on Testing Set on info gain')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Decision Trees using Gini impurity

In [ ]:
def decision_tree(X_train, y_train, remaining_attributes):
    # If all instances belong to the same class
    if len(set(y_train)) == 1:
        return {"class_label": y_train.iloc[0]}
    if not remaining_attributes:
        # If no more attributes left
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        return {"class_label": majority_class}


def calculate_gini_index(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    gini_index_value = 1 - np.square(probabilities).sum()
    return gini_index_value

def gini_attribute(y_train, attribute_values):
    gini = 0.0
    for value in attribute_values:
        partition_indices = y_train[attribute_values == value].index
        partition_gini = calculate_gini_index(y_train[partition_indices])
        gini += len(partition_indices) / len(y_train) * partition_gini
    return gini


In [ ]:
def predict_single(x, tree):
    if 'class_label' in tree:
        # Single data point
        return tree['class_label']
    else:
        value = x[tree['attribute']]
        if value in tree['leaf']:
            # leaf has only class label or a nested dictionary
            next_node = tree['leaf'][value]
            if 'class_label' in next_node:
                return next_node['class_label']
            else:
                return predict_single(x, next_node)
        else:
            return list(tree['leaf'].values())[0]


def predict(X, tree):
    return [predict_single(x, tree) for i, x in X.iterrows()]

In [ ]:
attributes = df_voting_shuffle.columns[:-1].tolist()
target = df_voting_shuffle['target']
train_df, test_df = train_test_split(df_voting_shuffle, test_size=0.2, random_state=34)

In [ ]:
 # Split the DataFrame into features and target for training and testing sets
X_train, y_train = train_df[attributes], train_df['target']
X_test, y_test = test_df[attributes], test_df['target']

In [ ]:
# Build the decision tree using the training set
root_node = decision_tree(X_train, y_train, attributes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# Function to calculate accuracy
def calculate_accuracy(predictions, actual):
    return np.sum(predictions == actual) / len(actual)

# Function to train and evaluate Decision Tree 100 times
def run_decision_tree(df, attributes, target, num_iterations=100):
    accuracy_train_list = []
    accuracy_test_list = []

    for i in range(num_iterations):
        # Shuffle the dataset
        df_shuffled = shuffle(df)

        # Split the DataFrame into features and target for training and testing sets
        train_df, test_df = train_test_split(df_shuffled, test_size=0.2, random_state=np.random.randint(1, 1000))

        X_train, y_train = train_df[attributes], train_df[target]
        X_test, y_test = test_df[attributes], test_df[target]

        # Build the decision tree using the training set
        root_node = decision_tree(X_train, y_train, attributes)

        # Make predictions on the training set
        predictions_train = predict(X_train, root_node)
        accuracy_train = calculate_accuracy(predictions_train, y_train)
        accuracy_train_list.append(accuracy_train)

        # Make predictions on the test set
        predictions_test = predict(X_test, root_node)
        accuracy_test = calculate_accuracy(predictions_test, y_test)
        accuracy_test_list.append(accuracy_test)

    return accuracy_train_list, accuracy_test_list


In [ ]:
# Run the experiment
attributes = df_voting_shuffle.columns[:-1].tolist()
target = 'target'
df_voting_shuffle = shuffle(df_voting_shuffle, random_state=np.random.randint(1, 1000)) 
accuracy_train, accuracy_test = run_decision_tree(df_voting_shuffle, attributes, target)

In [ ]:
mean_accuracy_train = np.mean(accuracy_train)
mean_accuracy_test = np.mean(accuracy_test)

# Print or use the values as needed
print("Mean Training Accuracy:", mean_accuracy_train)
print("Mean Testing Accuracy:", mean_accuracy_test)


std_dev_accuracy_train = np.std(accuracy_train)
std_dev_accuracy_test = np.std(accuracy_test)

print("Standard Deviation Training Accuracy:", std_dev_accuracy_train)
print("Standard Deviation Testing Accuracy:", std_dev_accuracy_test)

In [ ]:

# Plot histograms
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(accuracy_train, bins=20, edgecolor='black')
plt.title('Accuracy Distribution on Training Set on Gini')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')


plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(accuracy_test, bins=20, edgecolor='black')
plt.title('Accuracy Distribution on Testing Set on Gini')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Decision Trees with 0.85 threshold

In [ ]:
def decision_tree(X_train, y_train, remaining_attributes):
    # If all instances belong to the same class
    if len(set(y_train)) == 1:
        return {"class_label": y_train.iloc[0]}
    if not remaining_attributes or len(y_train) <= 0.85 * len(X_train):
        # If no more attributes left or instances threshold reached
        class_counts = y_train.value_counts()
        majority_class = class_counts.idxmax()
        return {"class_label": majority_class}

    info_gain_values = [information_gain(y_train, X_train[attr]) for attr in remaining_attributes]
    best_attribute = remaining_attributes[np.argmax(info_gain_values)] # Highest info gain attribute
    remaining_attributes_copy = remaining_attributes.copy()  # Make a copy
    remaining_attributes_copy.remove(best_attribute) # L(attr)-Selected(attr)
    remaining_attributes = remaining_attributes_copy.copy()  # Update remaining_attributes
    
    # Create a dictionary for the current decision node
    node = {"attribute": best_attribute, "leaf": {}}
    # Create a list of values for the selected attribute
    attribute_values = set(X_train[best_attribute])

    # For each attribute value, create sub-trees and edges
    for value in attribute_values:
        # Partition - based on the attribute value
        partition = X_train[X_train[best_attribute] == value]

        if partition.empty:
            # If partition is empty, leaf node = node with majority class
            majority_class = y_train.value_counts().idxmax()
            node["leaf"][value] = {"class_label": majority_class}
        else:
            # Recursively create a sub-tree
            subtree = decision_tree(partition, y_train[partition.index], remaining_attributes)
            node["leaf"][value] = subtree
    return node

def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def information_gain(y, x):
    parent_entropy = entropy(y)
    # InfoA(D)
    info_a = 0
    for value in set(x):
        partition_indices = x[x == value].index
        partition_entropy = entropy(y[partition_indices])
        info_a += len(partition_indices) / len(x) * partition_entropy
    # GainA(D)
    gain_a = parent_entropy - info_a
    return gain_a

In [ ]:
def predict_single(x, tree):
    if 'class_label' in tree:
        # Single data point
        return tree['class_label']
    else:
        value = x[tree['attribute']]
        if value in tree['leaf']:
            # leaf has only class label or a nested dictionary
            next_node = tree['leaf'][value]
            if 'class_label' in next_node:
                return next_node['class_label']
            else:
                return predict_single(x, next_node)
        else:
            return list(tree['leaf'].values())[0]

def predict(X, tree):
    return [predict_single(x, tree) for i, x in X.iterrows()]

In [ ]:
attributes = df_voting_shuffle.columns[:-1].tolist()
target = df_voting_shuffle['target']
train_df, test_df = train_test_split(df_voting_shuffle, test_size=0.2, random_state=34)

 # Split the DataFrame into features and target for training and testing sets
X_train, y_train = train_df[attributes], train_df['target']
X_test, y_test = test_df[attributes], test_df['target']

# Build the decision tree using the training set
root_node = decision_tree(X_train, y_train, attributes)

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(predictions, actual):
    return np.sum(predictions == actual) / len(actual)

# Function to train and evaluate Decision Tree 100 times
def run_decision_tree(df, attributes, target, num_iterations=100):
    accuracy_train_list = []
    accuracy_test_list = []

    for i in range(num_iterations):
        # Shuffle the dataset
        df_shuffled = shuffle(df)

        # Split the DataFrame into features and target for training and testing sets
        train_df, test_df = train_test_split(df_shuffled, test_size=0.2, random_state=np.random.randint(1, 1000))

        X_train, y_train = train_df[attributes], train_df[target]
        X_test, y_test = test_df[attributes], test_df[target]

        # Build the decision tree using the training set
        root_node = decision_tree(X_train, y_train, attributes)

        # Make predictions on the training set
        predictions_train = predict(X_train, root_node)
        accuracy_train = calculate_accuracy(predictions_train, y_train)
        accuracy_train_list.append(accuracy_train)

        # Make predictions on the test set
        predictions_test = predict(X_test, root_node)
        accuracy_test = calculate_accuracy(predictions_test, y_test)
        accuracy_test_list.append(accuracy_test)
        
    return accuracy_train_list, accuracy_test_list

In [ ]:
# Run the experiment
attributes = df_voting_shuffle.columns[:-1].tolist()
target = 'class'
df_voting_shuffle = shuffle(df_voting_shuffle, random_state=np.random.randint(1, 1000)) 
accuracy_train, accuracy_test = run_decision_tree(df_voting_shuffle, attributes, target)

In [ ]:
mean_accuracy_train = np.mean(accuracy_train)
mean_accuracy_test = np.mean(accuracy_test)

# Print or use the values as needed
print("Mean Training Accuracy:", mean_accuracy_train)
print("Mean Testing Accuracy:", mean_accuracy_test)


std_dev_accuracy_train = np.std(accuracy_train)
std_dev_accuracy_test = np.std(accuracy_test)

print("Standard Deviation Training Accuracy:", std_dev_accuracy_train)
print("Standard Deviation Testing Accuracy:", std_dev_accuracy_test)

In [ ]:
# Plot histograms
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(accuracy_train, bins=20, edgecolor='black')
plt.title('Accuracy Distribution on Training Set on 85% threshold')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')


In [ ]:
# Plot histograms
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 2)
plt.hist(accuracy_test, bins=20, edgecolor='black')
plt.title('Accuracy Distribution on Testing Set on 85% threshold')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()
